# ACL 2023
## Semantic Legal Searcher(SLS) : Neural Information Retrieval-based Semantic Search for Case Law

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/ACL_2023_SLS/

/content/drive/MyDrive/ACL_2023_SLS


## PIP

In [ ]:
! pip install transformers
! pip install -U sentence-transformers
! pip install sentencepiece
! pip install faiss-gpu
! pip install funcy pickle5

## STEP1. Load Dataframe & KRLawBERT

In [ ]:
import pandas as pd

# 1. Load Legal Dataset(Get Only Judicial precedent data)
df = pd.read_csv('./data/law_cases(20221020).csv')
df = df.dropna(how = 'any')

# 2. Make index for clustering
print(">> Law cases data size : ",len(df))
df.head()

>> Law cases data size :  20000


,case_name,case_number,date,case_code,judgment_issue,judgment_summary,judgment_contents,case_id,case_hits,case_hits_norm
1,지연이자청구,2015다223411,20170530.0,400101.0,"국가가 확정된 형사보상금의 지급을 지체하는 경우, 미지급 형사보상금에 대하여 지급...","헌법 제28조, 형사보상 및 명예회복에 관한 법률(이하 ‘형사보상법’이라고 한다) ...","【원고, 피상고인】 별지 원고 명단 기재와 같다. (소송대리인 동화법무법인 외 1인...",84878,8857,0.884545
3,손해배상(기)·매매대금반환,"2014다233176, 233183",20170530.0,400101.0,영국 계약법상 이행거절의 의미 및 이행거절의 의사를 표시했는지 판단하는 방법 / 당...,영국 계약법에서는 이행기 전 계약위반의 법리(doctrine of anticip...,"【원고(반소피고), 피상고인】 파산채무자 주식회사 21세기조선의 소송수계인 파산채무...",84876,3793,0.373030
4,숙박업영업신고증교부의무부작위위법확인,2017두34087,20170530.0,400107.0,숙박업을 하고자 하는 자가 법령이 정하는 시설과 설비를 갖추고 행정청에 신고를 ...,숙박업은 손님이 잠을 자고 머물 수 있도록 시설과 설비 등의 서비스를 제공하는 ...,"【원고, 상고인】 【피고, 피상고인】 속초시장【원심판결】 서울고법 2017. 1. ...",84875,7196,0.716768
5,손해배상(기),2017다211146,20170530.0,400101.0,새로운 청구를 심리하기 위하여 종전의 소송자료를 대부분 이용할 수 없고 별도의 증거...,민사소송법 제1조 제1항은 “법원은 소송절차가 공정하고 신속하며 경제적으로 진행되도...,"【원고, 상고인】 주식회사 메가럭 (소송대리인 법무법인(유한) 한동 담당변호사 고일...",84874,7832,0.781010
8,기타(금전),2016다275402,20170530.0,400101.0,"[1] 금전채무에 관하여 이행지체에 대비한 지연손해금 비율을 따로 약정한 경우, 민...",[1] 금전채무에 관하여 이행지체에 대비한 지연손해금 비율을 따로 약정한 경우에 이...,"【원고, 상고인】 현대건설 주식회사 (소송대리인 법무법인 가우 담당변호사 이경환 외...",84871,3410,0.334343


In [ ]:
# 3. Load pre-trained & fine-tuned models(KRLawBERT)
my_plms = './output/tsdae-krlawbert'

## STEP 2. Parallel clustering-based Topic Modeling

In [ ]:
from models.parallel_clustering_TM import *

In [ ]:
# 1. Obtain Legal Embeddings
target_text = 'judgment_summary'

cluster = ParallelCluster(
    dataframe = df,
    tgt_col = target_text,
    model_name = my_plms,
    use_sentence_bert = True
    )

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Data embeddings shape(Items x PLM_output) : (20000, 768)


In [ ]:
# 2. Parallel Clustering
clusters, unclusters = cluster.parallel_cluster(
    clusters = None,
    threshold = 0.71,
    page_size = 1000,
    iterations = 20
    )

===== Iteration 1 / 20 =====


>> Number of Total Clusters :  179
>> Percentage clusted Doc Embeddings : 31.72%


===== Iteration 2 / 20 =====


>> Number of Total Clusters :  401
>> Percentage clusted Doc Embeddings : 51.06%


===== Iteration 3 / 20 =====


>> Number of Total Clusters :  590
>> Percentage clusted Doc Embeddings : 61.19%


===== Iteration 4 / 20 =====


>> Number of Total Clusters :  728
>> Percentage clusted Doc Embeddings : 66.38%


===== Iteration 5 / 20 =====


>> Number of Total Clusters :  824
>> Percentage clusted Doc Embeddings : 69.06%


===== Iteration 6 / 20 =====


>> Number of Total Clusters :  887
>> Percentage clusted Doc Embeddings : 70.56%


===== Iteration 7 / 20 =====


>> Number of Total Clusters :  938
>> Percentage clusted Doc Embeddings : 71.68%


===== Iteration 8 / 20 =====


>> Number of Total Clusters :  968
>> Percentage clusted Doc Embeddings : 72.30%


===== Iteration 9 / 20 =====


>> Number of Total Clusters :  1006
>> Percentage clusted

In [ ]:
# 3. Stack : Stack the clustered results in order of cluster size
col_list = ['case_name', 'case_number', 'date', 'judgment_issue', 'judgment_summary']
new_df = cluster.cluster_stack(
    col_list = col_list,
    clusters = clusters,
    unclusters = unclusters
    )

# 4. Extract Keywords from each documents
top_n_words = cluster.extract_top_n_words_per_topic(
    dataframe = new_df,
    n = 10,
    en = False
    )
new_df['keywords'] = [', '.join(top_n_words[i]) for i in new_df['Topic'].values]

# 5. Save the Parallel Clusted Dataset 
new_df.to_csv("./data/clusted_df.csv", sep=',', na_rep="NaN")

## STEP 3. Embedding modelization(split-merge) and scoring(multi-interactions)

In [ ]:
from models.semantic_legal_searcher import *

In [ ]:
# Dataframe with keyword extraction from Parallel clustering-based TM
new_df = pd.read_csv('./data/clusted_df.csv')
#new_df.head()

,Unnamed: 0,case_name,case_number,date,judgment_issue,judgment_summary,Topic,keywords
0,82,업무상배임[역설계 등의 방법으로 입수 가능한 상태에 있는 회사의 정보를 무단으로 반...,2018도4794,20220630.0,회사 직원이 경쟁업체 또는 스스로의 이익을 위하여 이용할 의사로 무단으로 자료를...,회사 직원이 경쟁업체 또는 스스로의 이익을 위하여 이용할 의사로 무단으로 자료를...,0,"이익을, 회사의, 재산상, 사무를, 처리하는, 손해를, 타인의, 회사에, 재산상의,..."
1,118,영업양도무효확인·사해행위취소[주주가 영업양도의 무효확인을 구하는 사건],"2018다228462, 228479",20220609.0,[1] 주식회사의 주주가 직접 회사와 제3자의 거래관계에 개입하여 회사가 체결한...,[1] 주식회사의 주주는 주식의 소유자로서 회사의 경영에 이해관계를 가지고 있기...,0,"이익을, 회사의, 재산상, 사무를, 처리하는, 손해를, 타인의, 회사에, 재산상의,..."
2,134,사기미수,2022도1227,20220526.0,[1] 소송사기를 인정할 때 유의할 사항 / 소송사기가 성립하기 위한 요건 [...,[1] 소송사기는 법원을 기망하여 자기에게 유리한 판결을 얻음으로써 상대방의 재...,0,"이익을, 회사의, 재산상, 사무를, 처리하는, 손해를, 타인의, 회사에, 재산상의,..."
3,174,부당이득금,2017다278187,20220512.0,[1] 채무자가 아닌 위탁자가 타인의 채무를 담보하기 위하여 금전채권자를 우선수...,"[1] 위탁자가 금전채권을 담보하기 위하여 금전채권자를 우선수익자, 위탁자를 수...",0,"이익을, 회사의, 재산상, 사무를, 처리하는, 손해를, 타인의, 회사에, 재산상의,..."
4,204,분양대금반환등청구의소,2019다224726,20220428.0,[1] 불법행위로 인한 손해배상사건에서 피해자에게 손해의 발생이나 확대에 관하여...,[1] 불법행위로 인한 손해배상사건에서 피해자에게 손해의 발생이나 확대에 관하여...,0,"이익을, 회사의, 재산상, 사무를, 처리하는, 손해를, 타인의, 회사에, 재산상의,..."


In [ ]:
# 1. Obtain query, documents, keywords embeddings from KRLawBERT
my_plms = './output/tsdae-krlawbert'
sls = SLS(
    dataframe = new_df,
    doc_col = 'judgment_summary',
    key_col = 'keywords',
    model_name = my_plms,
    use_sentence_bert = True,
    split_and_merge = True,
    multi_inter = True,
    )

# 2. Build the Index
# (Strategy 1) : All Distance Metric
all_index = sls.all_distance_metric()
# (Strategy 2) : Restricted Distance Metric
restricted_index = sls.restricted_distance_metric(nlist = 200, nprobe = 6)

>> Split and Merage embeddings shape(Items x PLMs_dim) : (20000, 768)


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Keywords embeddings shape(Items x PLMs_dim) : (20000, 768)


## STEP 4. Semantic law case search with SLS

In [ ]:
# 3. Semantic case law search (Legal Question-Answering)
my_query = "마약 불법거래 및 운반에 관한 판례"

original_outputs, _ = sls.semantic_search(
    user_query = my_query,
    top_k = 5,
    index = all_index,
    print_results = True,
    )


 === Calculate run time : 43.5369 ms === 

>> Write your case : 마약 불법거래 및 운반에 관한 판례

 >> Top 1 - Case name (Number) : 압류처분무효확인 [검사가 전직 대통령(사망)에 대한 추징판결의 집행을 위하여 「공무원범죄에 관한 몰수 특례법」 제9조의2에 따라 신탁회사가 신탁 받은 부동산에 대해 한 압류처분의 항고소송 대상적격성, 위법성 등이 문제된 사안] (2019두63447)  
 | Cluster : 564 
 | Extracted keywords : 공무원범죄몰수법, 추징의, 불법재산, 재산형, 마약류, 추징을, 집행을, 제9조의2에, 마약거래방지법, 몰수 
 | Date : 20220728.0 | Judgment Issue :   [1] 공무원범죄에 관한 몰수 특례법 제9조의2에 의하여 제3자를 상대로 한 추징의 집행은 범인에 대한 추징의 집행을 전제로 하는지 여부(적극)  [2] 특정공무원범죄를 범한 범인이 사망한 경우, 범인에 대하여 재산형 등의 집행 및 공무원범죄에 관한 몰수 특례법 제9조의2에 따른 범인 외의 자를 상대로 한 추징의 집행을 할 수 있는지 여부(소극) / 이 경우 검사가 취할 조치  [3] 공무원범죄에 관한 몰수 특례법 제9조의2에 따라 추징의 집행을 받는 제3자가 검사의 처분이 부당함을 이유로 형사소송법 제489조에 따라 재판을 선고한 법원에 재판의 집행에 관한 이의를 신청할 수 있는지 여부(적극) 및 그와 별도로 행정소송법상 항고소송을 제기하여 처분의 위법성 여부를 다툴 수 있는지 여부(적극)  [4] 특정공무원범죄를 범한 범인이 그 정황을 아는 수탁자와 신탁계약을 체결하여 불법재산 등의 소유권을 신탁한 경우, 신탁법 제22조 제1항 본문의 적용이 배제되는지 여부(적극)  [5] 공무원범죄에 관한 몰수 특례법 제9조의2에서 정한 ‘범인 외의 자’, ‘취득’의 해석  
 | Judgment Summary :   [1] 공무원범죄에 관한 몰수 특례법(

## Compare Result
### 1. Without Multi-interactions mechanism(Single-interaction)

In [ ]:
# 1. Obtain query, documents, keywords embeddings from KRLawBERT
sls = SLS(
    dataframe = new_df,
    doc_col = 'judgment_summary',
    key_col = 'keywords',
    model_name = my_plms,
    use_sentence_bert = True,
    split_and_merge = False,
    multi_inter = False,
    )

# 2. Build the Index
all_index = sls.all_distance_metric()

# 3. Semantic case law search (Legal Question-Answering)
my_query = "마약 불법거래 및 운반에 관한 판례"
original_outputs, _ = sls.semantic_search(
    user_query = my_query,
    top_k = 5,
    index = all_index,
    print_results = True,
    )

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Documents embeddings shape(Items x PLMs_dim) : (20000, 768)


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Keywords embeddings shape(Items x PLMs_dim) : (20000, 768)

 === Calculate run time : 36.2847 ms === 

>> Write your case : 마약 불법거래 및 운반에 관한 판례

 >> Top 1 - Case name (Number) : 수뢰후부정처사(인정된 죄명 : 뇌물수수) (99도2530)  
 | Cluster : 242 
 | Extracted keywords : 피고인, 직무와, 직무, 뇌물을, 군사기밀의, 뇌물죄에서, 수수한, 수뢰자가, 군사상의, 공무원이 
 | Date : 19991109.0 | Judgment Issue : [1] 뇌물죄에 있어서 '직무'의 의미[2] 음주운전을 적발하여 단속에 관련된 제반 서류를 작성한 후 운전면허 취소업무를 담당하는 직원에게 이를 인계하는 업무를 담당하는 경찰관이 피단속자로부터 운전면허가 취소되지 않도록 하여 달라는 청탁을 받고 금원을 교부받은 경우, 뇌물수수죄가 성립한다고 한 사례[3] 법원이 공소장변경 없이 직권으로 공소장에 기재된 공소사실과 다른 범죄사실을 인정하기 위한 요건[4] 수뢰후부정처사죄로 기소된 공소사실을 공소장 변경 없이 뇌물수수죄로 인정할 수 있다고 한 사례 
 | Judgment Summary :   [1] 뇌물죄는 직무집행의 공정과 이에 대한 사회의 신뢰에 기하여 직무행위의 불가매수성을 그 직접의 보호법익으로 하고 있으므로 뇌물성은 의무위반 행위나 청탁의 유무 및 금품수수 시기와 직무집행 행위의 전후를 가리지 아니한다 할 것이고, 따라서 뇌물죄에서 말하는 '직무'에는 법령에 정하여진 직무뿐만 아니라 그와 관련 있는 직무, 과거에 담당하였거나 장래에 담당할 직무 외에 사무분장에 따라 현실적으로 담당하지 않는 직무라도 법령상 일반적인 직무권한에 속하는 직무 등 공무원이 그 직위에 따라 공무로 담당할 일체의 직무를 포함한다.  [2] 음주운전을 적발하여 단속에 관련된 제반 서류를 작성한 후 운전면허 취소업

### 2. Without Split-merge mechanism(Encoding documents-level)

In [ ]:
# 1. Obtain query, documents, keywords embeddings from KRLawBERT

my_plms = './output/tsdae-kolawbert'
sls = SLS(
    dataframe = new_df,
    doc_col = 'judgment_summary',
    key_col = 'keywords',
    model_name = my_plms,
    use_sentence_bert = True,
    split_and_merge = False,
    multi_inter = True,
    )

# 2. Build the Index
all_index = sls.all_distance_metric()

# 3. Semantic case law search (Legal Question-Answering)
my_query = "마약 불법거래 및 운반에 관한 판례"
original_outputs, _ = sls.semantic_search(
    user_query = my_query,
    top_k = 5,
    index = all_index,
    print_results = True,
    )

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Documents embeddings shape(Items x PLMs_dim) : (20000, 768)


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

>> Keywords embeddings shape(Items x PLMs_dim) : (20000, 768)

 === Calculate run time : 35.3644 ms === 

>> Write your case : 마약 불법거래 및 운반에 관한 판례

 >> Top 1 - Case name (Number) : 마약류불법거래방지에관한특례법위반(인정된 죄명: 마약류불법거래방지에관한특례법위반방조)ㆍ마약류관리에관한법률위반(향정) (2020도16369)  
 | Cluster : 564 
 | Extracted keywords : 공무원범죄몰수법, 추징의, 불법재산, 재산형, 마약류, 추징을, 집행을, 제9조의2에, 마약거래방지법, 몰수 
 | Date : 20210429.0 | Judgment Issue : 마약류 불법거래 방지에 관한 특례법 제6조를 위반하여 마약류를 수출입ㆍ제조ㆍ매매하는 행위 등을 업으로 하는 범죄행위의 정범이 그 범죄행위로 얻은 수익은 같은 법 제13조부터 제16조까지의 규정에 따라 몰수ㆍ추징의 대상이 되는지 여부(적극) / 위 정범으로부터 대가를 받고 판매할 마약을 공급하는 방법으로 위 범행을 용이하게 한 방조범이 정범의 위 범죄행위로 인한 수익을 정범과 공동으로 취득하였다고 평가할 수 없는 경우, 방조행위로 얻은 재산 등에 한하여 몰수, 추징할 수 있는지 여부(적극) 
 | Judgment Summary : 마약류 불법거래 방지에 관한 특례법(이하 ‘마약거래방지법’이라고 한다) 제6조를 위반하여 마약류를 수출입ㆍ제조ㆍ매매하는 행위 등을 업으로 하는 범죄행위의 정범이 그 범죄행위로 얻은 수익은 마약거래방지법 제13조부터 제16조까지의 규정에 따라 몰수ㆍ추징의 대상이 된다. 그러나 위 정범으로부터 대가를 받고 판매할 마약을 공급하는 방법으로 위 범행을 용이하게 한 방조범은 정범의 위 범죄행위로 인한 수익을 정범과 공동으로 취득하였다고 평가할 수 없다면 위 몰수ㆍ추징 규정에 의하여 정범과 같이 추징할 수는 없고, 그 방조범으로부터는 방